In [ ]:
import math
import random
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import csv
import pandas as pd
import os

# Data Refining

In [ ]:
def shift_and_interpolation(folder):
    
    for filename in os.listdir(folder):
        df= pd.read_csv(os.path.join(folder,filename),index_col=0,header=1)
        for column in df.columns:
            count_nan=0
            count=0
            index_first=1000000
            index_last=-1
            arr= df[column].to_numpy()

            for i in range(0,arr.shape[0]):
                if(np.isnan(arr[i])):
                    count_nan=count_nan+1
                else:
                    count=count+1
                    index_first=min(index_first,i)
                    index_last=max(index_last,i)

            if(count_nan!=arr.shape[0]):
                if(count_nan==arr.shape[0]-1):
                    for i in range(0,arr.shape[0]):
                        arr[i]=arr[index_first]
                else:
                    for i in range(0,index_first):
                        arr[i]=arr[index_first]
                    for i in range(index_last+1,arr.shape[0]):
                        arr[i]=arr[index_last]

                    list_x=[]
                    list_y=[]
                    for i in range(index_first,index_last+1):
                        if(np.isnan(arr[i])==0):
                            list_x.append(i)
                            list_y.append(arr[i])
                    fun=interp1d(list_x,list_y)
                    for i in range(index_first,index_last+1):
                        if(np.isnan(arr[i])):
                            arr[i]=fun(i)
                    df[column]=arr
            file="E:/train_B/"+filename
            df.to_csv(file)
    return     

In [ ]:
shift_and_interpolation('E:/t2')

In [ ]:
def fillcolumns(folder,mylist=[]):
    
    for filename in os.listdir(folder):
        df= pd.read_csv(os.path.join(folder,filename),index_col=0)
        i=0
        for column in df.columns:
            arr=df[column].to_numpy()
            for j in range(0,arr.shape[0]):
                if(np.isnan(arr[j])):
                    arr[j]=mylist[i]
            i=i+1
            df[column]=arr
            
        file="E:/train_filled_B/"+filename
        df.to_csv(file,index=False)
    return

In [ ]:
mylist=[80,100,37,116,78,58,18,1,-2.5,21.3,0.4,7.4,40,99,17,12,56,8.5,107,0.81,0.1,115,1.1,2,3.4,3.9,0.6,0.01,34,9.1,27.3,8.2,217,167,67,1,0,1,0,8,0]
fillcolumns('E:/train_A',mylist)
fillcolumns('E:/train_B',mylist)

# Adding NewFeatures

In [ ]:
def newfeatures(folder):
    for filename in os.listdir(folder):
        df= pd.read_csv(os.path.join(folder,filename))
        arr=df['SepsisLabel'].to_numpy()
        df=df.drop(['SepsisLabel'],axis=1)
        SOFA_Coagulation=[]
        SOFA_Liver=[]
        SOFA_Cardiovascular=[]
        SOFA_Renal=[]

        Q_SOFA=[]

        platelets=df['Platelets'].to_numpy()
        bilirubin=df['Bilirubin_total'].to_numpy()
        cardio=df['MAP'].to_numpy()
        renal=df['Creatinine'].to_numpy()

        temp=df['Temp'].to_numpy()
        HR=df['HR'].to_numpy()
        resp=df['Resp'].to_numpy()
        paco2=df['PaCO2'].to_numpy()
        wbc=df['WBC'].to_numpy()

        for i in range (0,df.shape[0]):
            if(platelets[i]>=150):
                SOFA_Coagulation.append(0)
            if(platelets[i]<150 and platelets[i]>=100 ):
                SOFA_Coagulation.append(1)
            if(platelets[i]<100 and platelets[i]>=50):
                SOFA_Coagulation.append(2)
            if(platelets[i]<50 and platelets[i]>=20):
                SOFA_Coagulation.append(3)
            if(platelets[i]<20):
                SOFA_Coagulation.append(4)

            if(cardio[i]>=70):
                SOFA_Cardiovascular.append(0)
            else:
                SOFA_Cardiovascular.append(1)


            if(bilirubin[i]<1.2):
                SOFA_Liver.append(0)
            if(bilirubin[i]>=1.2 and bilirubin[i]<=2):
                SOFA_Liver.append(1)
            if(bilirubin[i]>2 and bilirubin[i]<6):
                SOFA_Liver.append(2)
            if(bilirubin[i]>=6 and bilirubin[i]<12):
                SOFA_Liver.append(3)
            if(bilirubin[i]>=12):
                SOFA_Liver.append(4)

            if(renal[i]<=1.2):
                SOFA_Renal.append(0)
            if(renal[i]>1.2 and renal[i]<2):
                SOFA_Renal.append(1)
            if(renal[i]>=2 and renal[i]<3.5):
                SOFA_Renal.append(2)
            if(renal[i]>=3.5 and renal[i]<5):
                SOFA_Renal.append(3)
            if(renal[i]>=5):
                SOFA_Renal.append(4)

            p=0
            q=0
            r=0
            if(temp[i]>38 and temp[i]<36):
                p=1
            if(HR[i]>90):
                q=1
            if(resp[i]>20 or resp[i]<32):
                r=1
            if(p+q+r>=2):
                Q_SOFA.append(1)
            else:
                Q_SOFA.append(0)
        df['SOFA_Coagulation']=SOFA_Coagulation
        df['SOFA_Liver']=SOFA_Liver
        df['SOFA_Cardiovascular']=SOFA_Cardiovascular
        df['SOFA_Renal']=SOFA_Renal
        df['Q_SOFA']=Q_SOFA
        df['SepsisLabel']=arr
        file="E:/train_new_A/"+filename
        df.to_csv(file,index=False)
    return

In [ ]:
newfeatures('E:/train_filled_A')

In [ ]:
newfeatures('E:/train_filled_B')

# Combining to Single Csv

In [ ]:
super_x = []
i=0
for filename in os.listdir('E:/train_new_B'):
    df1= pd.read_csv(os.path.join('E:/train_new_B',filename))
    
    super_x.append(df1)
    print(i)
    i=i+1
df = pd.concat(super_x, axis=0)
df.to_csv('B_ultimate.csv',index=False)

In [ ]:
data=pd.read_csv('A_ultimate.csv')
df1=pd.read_csv('B_ultimate.csv')
df=pd.concat([data,df1],axis=0,ignore_index=False)

# Normalizing Dataset

In [ ]:
data=pd.read_csv('A_ultimate.csv')
df1=pd.read_csv('B_ultimate.csv')
df=pd.concat([data,df1],axis=0,ignore_index=False)

arr1=df['Gender'].to_numpy()
arr2=df['Unit1'].to_numpy()
arr3=df['Unit2'].to_numpy()
arr4=df['SOFA_Coagulation'].to_numpy()
arr5=df['SOFA_Liver'].to_numpy()
arr6=df['SOFA_Cardiovascular'].to_numpy()
arr7=df['SOFA_Renal'].to_numpy()
arr8=df['Q_SOFA'].to_numpy()
arr9=df['SepsisLabel'].to_numpy()

In [ ]:
df1=df.drop(['Gender','Unit1','Unit2','SOFA_Coagulation','SOFA_Liver','SOFA_Cardiovascular','SOFA_Renal','Q_SOFA','SepsisLabel'],axis=1)
df1.head()

In [ ]:
normalized_df=(df1-df1.mean())/df1.std()

In [ ]:
normalized_df['Gender']=arr1
normalized_df['Unit1']=arr2
normalized_df['Unit2']=arr3
normalized_df['SOFA_Coagulation']=arr4
normalized_df['SOFA_Liver']=arr5
normalized_df['SOFA_Cardiovascular']=arr6
normalized_df['SOFA_Renal']=arr7
normalized_df['Q_SOFA']=arr8
normalized_df['SepsisLabel']=arr9

In [ ]:
normalized_df.to_csv('normalized_total.csv')